In [49]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn import ensemble
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

Perceptron

In [19]:
class PerceptronClass(object):
    """Perceptron classifier.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.

    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    errors_ : list
      Number of misclassifications (updates) in each epoch.

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          Training vectors, where n_samples is the number of samples and
          n_features is the number of features.
        y : array-like, shape = [n_samples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)

# Adaline SGD

In [42]:
class AdalineSGD(object):
    """ADAptive LInear NEuron classifier.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    shuffle : bool (default: True)
      Shuffles training data every epoch if True to prevent cycles.
    random_state : int
      Random number generator seed for random weight
      initialization.


    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    cost_ : list
      Sum-of-squares cost function value averaged over all
      training samples in each epoch.

        
    """
    def __init__(self, eta=0.01, n_iter=10, shuffle=True, random_state=None, batch_size=20):
        self.eta = eta
        self.n_iter = n_iter
        self.w_initialized = False
        self.shuffle = shuffle
        self.random_state = random_state
        self.batch_size = batch_size
        
    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          Training vectors, where n_samples is the number of samples and
          n_features is the number of features.
        y : array-like, shape = [n_samples]
          Target values.

        Returns
        -------
        self : object

        """
        self._initialize_weights(X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            cost = []
            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)
        return self

    def partial_fit(self, X, y):
        self._initialize_weights(X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            y = y.reshape(-1, 1)
            batches_X = [X[k:k+self.batch_size] for k in range(0, X.shape[0], self.batch_size)]
            batches_y = [y[k:k+self.batch_size] for k in range(0, y.shape[0], self.batch_size)]
            cost = []
            for Xi, target in zip(batches_X, batches_y):
                cost.append(self._update_weights(Xi, target))
            avg_cost = sum(cost) / len(batches_y)
            self.cost_.append(avg_cost)
        return self


    def _shuffle(self, X, y):
        """Shuffle training data"""
        r = self.rgen.permutation(len(y))
        return X[r], y[r]
    
    def _initialize_weights(self, m):
        """Initialize weights to small random numbers"""
        self.rgen = np.random.RandomState(self.random_state)
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=1 + m)
        self.w_initialized = True
        
    def _update_weights(self, xi, target):
        """Apply Adaline learning rule to update the weights"""
        output = self.activation(self.net_input(xi))
        error = (target - output)
        self.w_[1:] += self.eta * xi.dot(error)
        self.w_[0] += self.eta * error
        cost = (0.5) * np.square(error)
        return cost
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        """Compute linear activation"""
        return X

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.activation(self.net_input(X)) >= 0.0, 1, -1)

Adaline GD

In [53]:
class AdalineGD(object):
    """ADAptive LInear NEuron classifier.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.


    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    cost_ : list
      Sum-of-squares cost function value in each epoch.

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          Training vectors, where n_samples is the number of samples and
          n_features is the number of features.
        y : array-like, shape = [n_samples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.cost_ = []
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        for i in range(self.n_iter):
            net_input = self.net_input(X)
            # Please note that the "activation" method has no effect
            # in the code since it is simply an identity function. We
            # could write `output = self.net_input(X)` directly instead.
            # The purpose of the activation is more conceptual, i.e.,  
            # in the case of logistic regression (as we will see later), 
            # we could change it to
            # a sigmoid function to implement a logistic regression classifier.
            output = self.activation(net_input)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            squared_errors = np.float128(errors)**2
            cost = 0.5 * squared_errors.sum()
            cost = cost/2.0
            self.cost_.append(cost.astype(np.float64))
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        """Compute linear activation"""
        return X

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.activation(self.net_input(X)) >= 0.0, 1, -1)

In [63]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [62]:
Perceptron_obj = PerceptronClass()
Perceptron_skt = Perceptron(tol=1e-3, random_state=0)

Perceptron_skt.fit(X,y)
Perceptron_obj.fit(X,y)

y_pred_obj=Perceptron_obj.predict(X_test)
y_pred_skt=Perceptron_skt.predict(X_test)



print("Sklearn: "+str(metrics.accuracy_score(y_test,y_pred_skt)))


print("Class: "+str(metrics.accuracy_score(y_test,y_pred_obj)))

Sklearn: 0.9440559440559441
Class: 0.6293706293706294


In [61]:
SGD_SKT = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
SGD_SKT.fit(X,y)

AdalineSGD_obj = AdalineSGD()
AdalineSGD_obj.fit(X,y)

y_pred_obj = AdalineSGD_obj.predict(X_test)
y_pred_skt = SGD_SKT.predict(X_test)

print("Sktlearn: "+str(metrics.accuracy_score(y_test,y_pred_skt)))
print("AdalineSGD: "+str(metrics.accuracy_score(y_test,y_pred_obj)))



Sktlearn: 0.986013986013986
AdalineSGD: 0.0


<ipython-input-42-cd576667d06e>:97: RuntimeWarning: overflow encountered in square
  cost = (0.5) * np.square(error)
<ipython-input-42-cd576667d06e>:95: RuntimeWarning: invalid value encountered in add
  self.w_[1:] += self.eta * xi.dot(error)
<ipython-input-42-cd576667d06e>:96: RuntimeWarning: invalid value encountered in double_scalars
  self.w_[0] += self.eta * error


Deu um erro a na hora de utilizar a classe do AdalineSGD e não consegui identificar.


In [60]:
gd_skt = LogisticRegression(random_state=0)
gd_obj = AdalineGD()


gd_skt.fit(X,y)
gd_obj.fit(X,y)


y_pred_obj = gd_obj.predict(X_test)
y_pred_skt = gd_skt.predict(X_test)

print("Sktlearn: "+str(metrics.accuracy_score(y_test,y_pred_skt)))
print("AdalineSGD: "+str(metrics.accuracy_score(y_test,y_pred_obj)))

Sktlearn: 0.951048951048951
AdalineSGD: 0.6293706293706294


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
